<a href="https://colab.research.google.com/github/vinaykotadiya/ev-adoption-forecast/blob/main/ZEV_Forcasting_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import library
import pandas as pd

In [2]:
# read the uploaded file
df = pd.read_csv('df_VEH0120_UK.csv')

# show basic info
print("Rows and Columns:", df.shape)
df.head()

Rows and Columns: (208751, 49)


,BodyType,Make,GenModel,Model,Fuel,LicenceStatus,2025Q1,2024Q4,2024Q3,2024Q2,...,2016Q4,2016Q3,2016Q2,2016Q1,2015Q4,2015Q3,2015Q2,2015Q1,2014Q4,2014Q3
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,Licensed,15,14,15,16,...,0,0,0,0,0,0,0,0,0,0
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,SORN,4,5,4,3,...,0,0,0,0,0,0,0,0,0,0
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,Licensed,26,25,27,25,...,0,0,0,0,0,0,0,0,0,0
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,SORN,3,4,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Cars,ABARTH,ABARTH SPIDER,124 SPIDER MULTIAIR,Petrol,Licensed,973,960,978,978,...,155,100,0,0,0,0,0,0,0,0


In [3]:
# check for missing values and duplicates
print("\nMissing values per column:\n", df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())


Missing values per column:
 BodyType         0
Make             0
GenModel         2
Model            0
Fuel             0
LicenceStatus    0
2025Q1           0
2024Q4           0
2024Q3           0
2024Q2           0
2024Q1           0
2023Q4           0
2023Q3           0
2023Q2           0
2023Q1           0
2022Q4           0
2022Q3           0
2022Q2           0
2022Q1           0
2021Q4           0
2021Q3           0
2021Q2           0
2021Q1           0
2020Q4           0
2020Q3           0
2020Q2           0
2020Q1           0
2019Q4           0
2019Q3           0
2019Q2           0
2019Q1           0
2018Q4           0
2018Q3           0
2018Q2           0
2018Q1           0
2017Q4           0
2017Q3           0
2017Q2           0
2017Q1           0
2016Q4           0
2016Q3           0
2016Q2           0
2016Q1           0
2015Q4           0
2015Q3           0
2015Q2           0
2015Q1           0
2014Q4           0
2014Q3           0
dtype: int64

Duplicate rows: 0


In [4]:
# remove duplicate and empty rows
df.drop_duplicates(inplace=True)
df.dropna(how='all', inplace=True)

In [5]:
# clean text data (remove spaces and fix case)
df['BodyType'] = df['BodyType'].str.strip().str.title()
df['Fuel'] = df['Fuel'].str.strip().str.title()
df['LicenceStatus'] = df['LicenceStatus'].str.strip().str.title()

In [6]:
# show unique values to understand data categories
print("\nDifferent BodyType:", df['BodyType'].unique())
print("\nDifferent Fuel:", df['Fuel'].unique())
print("\nDifferent LicenceStatus:", df['LicenceStatus'].unique())


Different BodyType: ['Cars' 'Heavy Goods Vehicles' 'Light Goods Vehicles' 'Other Vehicles'
 'Motorcycles' 'Buses And Coaches']

Different Fuel: ['Petrol' 'Battery Electric' 'Gas' 'Diesel' 'Hybrid Electric (Petrol)'
 'Plug-In Hybrid Electric (Petrol)' 'Hybrid Electric (Diesel)'
 'Plug-In Hybrid Electric (Diesel)' 'Other Fuel Types'
 'Range Extended Electric' 'Fuel Cell Electric']

Different LicenceStatus: ['Licensed' 'Sorn']


In [7]:
# filter only licensed fully electric cars
ev_types = ['Battery Electric', 'Fuel Cell Electric', 'Range Extended Electric']

df_cars = df[
    (df['BodyType'] == 'Cars') &
    (df['Fuel'].isin(ev_types)) &
    (df['LicenceStatus'] == 'Licensed')
].reset_index(drop=True)

In [8]:
# preview the filtered dataset
print(df_cars.shape)
df_cars.head()

(1359, 49)


,BodyType,Make,GenModel,Model,Fuel,LicenceStatus,2025Q1,2024Q4,2024Q3,2024Q2,...,2016Q4,2016Q3,2016Q2,2016Q1,2015Q4,2015Q3,2015Q2,2015Q1,2014Q4,2014Q3
0,Cars,ABARTH,ABARTH 500,500E,Battery Electric,Licensed,232,192,139,68,...,0,0,0,0,0,0,0,0,0,0
1,Cars,ABARTH,ABARTH 500,500E SCORPIONISSIMA,Battery Electric,Licensed,133,130,129,111,...,0,0,0,0,0,0,0,0,0,0
2,Cars,ABARTH,ABARTH 500,500E TURISMO,Battery Electric,Licensed,182,177,164,140,...,0,0,0,0,0,0,0,0,0,0
3,Cars,ABARTH,ABARTH 600E,600E,Battery Electric,Licensed,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cars,ABARTH,ABARTH 600E,600E SCORPIONISSIMA,Battery Electric,Licensed,26,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
print("\nDifferent BodyType:", df_cars['BodyType'].unique())
print("\nDifferent Fuel:", df_cars['Fuel'].unique())
print("\nDifferent LicenceStatus:", df_cars['LicenceStatus'].unique())


Different BodyType: ['Cars']

Different Fuel: ['Battery Electric' 'Range Extended Electric' 'Fuel Cell Electric']

Different LicenceStatus: ['Licensed']


In [10]:
# save clean dataset
df_cars.to_csv('ev_cars_uk.csv', index=False)